In [ ]:
# @title **セットアップ**

! pip install transformers gradio scipy ftfy "ipywidgets>=7,<8" datasets

! git clone https://github.com/huggingface/diffusers.git
! pip install git+https://github.com/huggingface/diffusers.git
%cd diffusers

In [ ]:
# @title **ログイン**
# @markdown - 事前にHagging Faceでトークンを取得しておいて下さい
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# @title **Pipeline**
import torch
from diffusers import StableDiffusionPipeline
from examples.inference.image_to_image import StableDiffusionImg2ImgPipeline, preprocess

pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True,
).to("cuda")
pipeimg = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True,
).to("cuda")

In [ ]:
# @title **メインプログラム**
import gradio as gr
import torch
from torch import autocast
from diffusers import LMSDiscreteScheduler
import requests
from PIL import Image
from io import BytesIO
from IPython.display import clear_output

from examples.inference.image_to_image import preprocess

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")

def process(prompt, num_of_images, num_inference_steps, guidance_scale, init_image, strength):
    num_of_images = int(num_of_images)
    if init_image != None:
        init_image = init_image.resize((512, 512))
        init_image = preprocess(init_image)
        with autocast("cuda"):
            images = pipeimg(
                [prompt] * num_of_images,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                init_image=init_image,
                strength=strength,
            )["sample"]
    else:
        with autocast("cuda"):
            images = pipe(
                [prompt] * num_of_images,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                init_image=init_image,
            )["sample"]

    return images


with block as demo:
    gr.Markdown("<h1>Stable Diffusion UI</h1>")
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):

                text = gr.Textbox(
                    label="Enter your prompt", show_label=False
                ).style(
                    border=True,
                    rounded=True,
                    container=False,
                )
                btn = gr.Button("Run").style(
                    rounded=True,
                )
        num_of_images = gr.Number(label="Number of images", value=1)
        num_inference_steps = gr.Slider(label="Steps", maximum=150, value=50, step=1)
        guidance_scale = gr.Slider(label="Cfg Scale", maximum=20, value=7.5, step=0.5)
        image = gr.Image(label="Intial Image", type="pil")
        strength_slider = gr.Slider(label="Strength", maximum=1, value=0.75)

        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[2], height="auto"
        )
        btn.click(
            process,
            inputs=[text, num_of_images, num_inference_steps, guidance_scale, image, strength_slider],
            outputs=gallery,
        )

clear_output()
demo.launch(debug=True)